# Introduction to the Data.

In this notebook we are going to clean a CSV dataset and add it to a SQLite database. The data contains information on Academy Awards nominations, also known as Oscars, and it is broke down in the following columns:

· __Year__: the year of the awards ceremony.

· __Category__: the category of award the nominee was nominated for.

· __Nominee__: the person nominated for the award.     

· __Additional info__: it contains additional info like the movie the nominee participated or the name of the character played.

· __Won?__: YES or NO depending on if the nominee won the award.
    
The dataset we will use can be found here: https://www.aggdata.com/awards/oscar

In [108]:
# We will start importing pandas and reading our file into a Dataframe, 

import pandas as pd
academy_awards = pd.read_csv('academy_awards.csv',encoding = 'ISO-8859-1')
academy_awards.head(5)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


The first thing we can see when exploring the first rows of the Dataframe is that we have 6 *unnamed columns*, full of NaNs. To see if there are different values, the *value_counts* method will be used in every column.

In [109]:
academy_awards['Unnamed: 5'].value_counts(normalize = True, dropna = False)

NaN                                                                                                             0.998915
*                                                                                                               0.000691
 discoverer of stars                                                                                            0.000099
 error-prone measurements on sets. [Digital Imaging Technology]"                                                0.000099
 resilience                                                                                                     0.000099
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    0.000099
Name: Unnamed: 5, dtype: float64

In [110]:
academy_awards['Unnamed: 6'].value_counts(normalize = True, dropna = False)

NaN                                                                 0.998816
*                                                                   0.000888
 sympathetic                                                        0.000099
 direct radiator bass style cinema loudspeaker systems. [Sound]"    0.000099
 flexibility and water resistance                                   0.000099
Name: Unnamed: 6, dtype: float64

In [111]:
academy_awards['Unnamed: 7'].value_counts(normalize = True, dropna = False)

NaN                                                   0.999704
*                                                     0.000099
 while requiring no dangerous solvents. [Systems]"    0.000099
 kindly                                               0.000099
Name: Unnamed: 7, dtype: float64

In [112]:
academy_awards['Unnamed: 8'].value_counts(normalize = True, dropna = False)

NaN                                               0.999803
*                                                 0.000099
 understanding comedy genius - Mack Sennett.""    0.000099
Name: Unnamed: 8, dtype: float64

In [113]:
academy_awards['Unnamed: 9'].value_counts(normalize = True, dropna = False)

NaN    0.999901
*      0.000099
Name: Unnamed: 9, dtype: float64

In [114]:
academy_awards['Unnamed: 10'].value_counts(normalize = True, dropna = False)

NaN    0.999901
*      0.000099
Name: Unnamed: 10, dtype: float64

# Filtering the Data

As we could see in the exploratory analysis this dataset doesn't have a consistent format, which is extremely important to query the data later on when we use SQL.

To make our dataset easier to work with, we are going to make some filtering to get a smaller subset that only contains the next values for the "Category" column:

· __Actor -- Leading Role.__

· __Actor -- Supporting Role.__

· __Actress -- Leading Role.__

· __Actress -- Supporting Role.__

First, we are going to clean up the "Year" column, because we just want the value outside of parentheses and converted to integers, so we can handle them better. Then, we will build a dataset whose values for the "Year" column will always be greater than 2000 and use those rows together with the four categories of the "Category" column mentioned above to create a new dataset named nominations.

In [115]:
academy_awards["Year"] = academy_awards["Year"].str[0:4].astype('int64')

In [116]:
later_than_2000 = academy_awards[academy_awards["Year"] > 2000]

In [124]:
award_categories = ['Actor -- Leading Role', 'Actor -- Supporting Role', 'Actress -- Leading Role', 'Actress -- Supporting Role']

nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]

In [125]:
nominations.head(5)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning up problematic columns

SQLite use integers 0 and 1 to represent Boolean values, therefore, our column "Won?" might be problematic in the future if we don't convert its values. Since we have to convert its values, we will also change its name to just "Won".

To carry out the first task, we will use a map function and a dictionary

In [126]:
mapper = { "NO": 0, "YES": 1 }

nominations["Won?"] = nominations["Won?"].map({ "NO": 0, "YES": 1 })

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [127]:
nominations["Won"] = nominations["Won?"]
nominations["Won"]

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0       0
1       0
2       0
3       1
4       0
5       1
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      1
14      0
15      0
16      0
17      1
18      0
19      0
135     1
136     0
137     0
138     0
139     0
140     0
141     0
142     0
143     0
144     1
       ..
1040    0
1041    1
1042    0
1043    0
1044    0
1045    0
1046    0
1047    0
1048    0
1049    1
1157    0
1158    0
1159    0
1160    1
1161    0
1162    1
1163    0
1164    0
1165    0
1166    0
1167    1
1168    0
1169    0
1170    0
1171    0
1172    1
1173    0
1174    0
1175    0
1176    0
Name: Won, dtype: int64

We now have two columns representing nominations["Won"]. We want to get rid of several columns like "Won?" and the "Unnamed" ones. So we will use the *drop* method to remove our extra columns.

In [128]:
extra_columns = ['Won?', 
                 'Unnamed: 5', 
                 'Unnamed: 6', 
                 'Unnamed: 7', 
                 'Unnamed: 8', 
                 'Unnamed: 9', 
                 'Unnamed: 10']

final_nominations = nominations.drop(extra_columns, axis= 1)

# Cleaning up the "Additional Info" Column

Our "Additional Info" column is formatted for every nominee so that it contains the name of the movie and the character played by the nominee. For example:

The Social Network {'Mark Zuckerberg'}

To clean this column, we will use the *rstrip* method.

In [144]:
additional_info_one = final_nominations['Additional Info'].str.rstrip("'}")

In [ ]:
additional_info_two = additional_info_one.str.split(" {'")

In [151]:
movie_names = additional_info_two.str[0]
characters= additional_info_two.str[1]
final_nominations['Movie'] = movie_names
final_nominations['Character'] = characters
final_nominations = final_nominations.drop('Additional Info', axis = 1)
final_nominations.head(5)

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston


# Exporting to SQLite

Now that our Dataframe is cleaned up, we can write its record to a SQL database. To accomplish it, we will use the *to_sql* method. Since we are creating a database from zero, we need to create a database file first using *sqlite3* to connect to a file path that doesn't exist yet (python will create it for us).

Let's create the SQLite database and connect to it

In [153]:
import sqlite3 as sql

conn = sql.connect('nominations.db') 

In [156]:
final_nominations.to_sql(name = 'nominations', con = conn, index = False)

# Verifying in SQL

Let's query our database to make sure everything worked as expected.

In [157]:
conn2 = sql.connect('nominations.db')

In [164]:
query1 = "pragma table_info(nominations)"
print(conn2.execute(query1).fetchall())

[(0, 'Year', 'INTEGER', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Nominee', 'TEXT', 0, None, 0), (3, 'Won', 'INTEGER', 0, None, 0), (4, 'Movie', 'TEXT', 0, None, 0), (5, 'Character', 'TEXT', 0, None, 0)]


In [165]:
query2 = "SELECT * FROM nominations LIMIT 10"
print(conn2.execute(query1).fetchall())

[(0, 'Year', 'INTEGER', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Nominee', 'TEXT', 0, None, 0), (3, 'Won', 'INTEGER', 0, None, 0), (4, 'Movie', 'TEXT', 0, None, 0), (5, 'Character', 'TEXT', 0, None, 0)]
